In [51]:
import pandas as pd
import numpy as np
import os
from functools import reduce
from tabulate import tabulate

In [52]:
total = []
std = pd.read_csv('../Data/Intermediate/COVIDcounty_cases_processed_2020.csv', index_col = 0)
race = pd.read_csv('../Data/Clean/nonACSDemographics/census_race_sex_ethnicity_pop_2000_2010_clean.csv', index_col = 0)
dp05 = pd.read_csv('../Data/Intermediate/ACSDP05_merge.csv', index_col = 0)
dp03 = pd.read_csv('../Data/Intermediate/ACSDP03_merge.csv', index_col = 0)
saipe = pd.read_csv('../Data/Intermediate/saipe0210.csv', index_col = 0)
hps = pd.read_csv('../Data/Intermediate/hps_processed.csv', index_col = 0)
total.extend([std, dp05, dp03, saipe])
#print(total)

In [53]:
for index, i in enumerate(total):
    i['fips'] = i['fips'].astype(str).str.zfill(5)
    print(i.shape)
    if 'year' in i.columns:
        if 'area' in i.columns:
            i.drop(columns = 'area', inplace = True)
            print(index, ': ', 'Done')
        if 'county_name' in i.columns:
            i.drop(columns = 'county_name' , inplace = True)
            print(index, ': ', 'done')
        if 'state_fips' in i.columns:
            i.drop(columns = 'state_fips', inplace = True)
            print(index, ': ', 'DONE')

(3142, 4)
0 :  done
(35425, 11)
(35425, 20)
(32280, 15)


In [54]:
cov = total[0].columns.intersection(total[1].columns)
df = pd.merge(total[0], total[1], on = cov.to_list(), how = 'outer')
df = df[(df['year'] >= 2002) & (df['year'] <= 2020)]
df = df.sort_values(by = ['fips','year']).reset_index(drop = True)
print(df['year'].value_counts())
print(df.shape)
df.head(20)

2020    3222
2010    3221
2011    3221
2012    3221
2013    3221
2014    3220
2015    3220
2016    3220
2017    3220
2018    3220
2019    3220
Name: year, dtype: int64
(35426, 12)


,year,fips,covid_cases,total_pop,pct_female,pct_white,pct_black,pct_native_american,pct_asian,pct_native_hawaiian_pacific_islander,pct_two_or_more_races,pct_hispanic
0,2010,01001,NaN,53155.0,0.515,0.791,0.179,0.003,0.007,0.000,0.015,0.023
1,2011,01001,NaN,53944.0,0.515,0.789,0.181,0.002,0.007,0.000,0.015,0.024
2,2012,01001,NaN,54590.0,0.514,0.789,0.181,0.003,0.008,0.000,0.015,0.024
3,2013,01001,NaN,54907.0,0.512,0.783,0.184,0.003,0.010,0.000,0.014,0.025
4,2014,01001,NaN,55136.0,0.514,0.780,0.186,0.004,0.009,0.000,0.013,0.025
5,2015,01001,NaN,55221.0,0.516,0.774,0.187,0.004,0.010,0.000,0.016,0.026
6,2016,01001,NaN,55049.0,0.512,0.769,0.185,0.004,0.008,0.001,0.020,0.026
7,2017,01001,NaN,55036.0,0.511,0.766,0.191,0.003,0.009,0.001,0.017,0.027
8,2018,01001,NaN,55200.0,0.513,0.769,0.191,0.003,0.010,0.001,0.019,0.028
9,2019,01001,NaN,55380.0,0.514,0.768,0.190,0.003,0.010,0.000,0.022,0.028


In [55]:
for i in total[2:9]:
    cov = df.columns.intersection(i.columns)
    df = pd.merge(df, i, on = cov.to_list(), how = 'outer')
    df = df[(df['year'] >= 2002) & (df['year'] <= 2020)]
    df = df.sort_values(by = ['fips','year']).reset_index(drop = True)
    print(df['year'].value_counts())
    print(df.shape)

2020    3222
2010    3221
2011    3221
2012    3221
2013    3221
2014    3220
2015    3220
2016    3220
2017    3220
2018    3220
2019    3220
Name: year, dtype: int64
(35426, 30)
2002    3228
2003    3228
2004    3228
2005    3228
2006    3228
2007    3228
2008    3228
2009    3228
2020    3222
2010    3221
2012    3221
2013    3221
2011    3221
2014    3220
2015    3220
2016    3220
2017    3220
2018    3220
2019    3220
Name: year, dtype: int64
(61250, 31)


In [56]:
print(df.dtypes)
df.head(30)

year                                                                                                 int64
fips                                                                                                object
covid_cases                                                                                        float64
total_pop                                                                                          float64
pct_female                                                                                         float64
pct_white                                                                                          float64
pct_black                                                                                          float64
pct_native_american                                                                                float64
pct_asian                                                                                          float64
pct_native_hawaiian_pacific_islander 

,year,fips,covid_cases,total_pop,pct_female,pct_white,pct_black,pct_native_american,pct_asian,pct_native_hawaiian_pacific_islander,...,pct_ind_finance_and_insurance_and_real_estate_and_rental_and_leasing,pct_ind_professional_scientific_and_management_and_administrative_and_waste_management_services,pct_ind_educational_services_and_health_care_and_social_assistance,pct_ind_arts_entertainment_and_recreation_and_accommodation_and_food_services,pct_ind_other_services_except_public_administration,pct_ind_public_administration,health_insurance_rate,poverty_rate,per_capita_income_2022,employed_persons
0,2002,01001,NaN,45909.0,0.512906,0.807445,0.172123,0.004792,0.006055,0.000283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.3,70687.65,21096.0
1,2003,01001,NaN,46800.0,0.513675,0.805897,0.172842,0.004808,0.006282,0.000321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.4,71228.01,21557.0
2,2004,01001,NaN,48366.0,0.513873,0.804139,0.173531,0.004673,0.006740,0.000351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.6,70791.24,22146.0
3,2005,01001,NaN,49676.0,0.512843,0.802440,0.173706,0.004549,0.007247,0.000745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.4,68428.88,22986.0
4,2006,01001,NaN,51328.0,0.513170,0.799388,0.176181,0.004481,0.007286,0.000760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5,67876.86,23619.0
5,2007,01001,NaN,52405.0,0.514016,0.797328,0.177273,0.004446,0.007709,0.000725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.4,71532.50,23610.0
6,2008,01001,NaN,53277.0,0.514368,0.795071,0.178126,0.004730,0.008033,0.000845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7,69689.70,23376.0
7,2009,01001,NaN,54135.0,0.512995,0.793091,0.179071,0.004674,0.008497,0.000868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.2,72720.97,22464.0
8,2010,01001,NaN,53155.0,0.515000,0.791000,0.179000,0.003000,0.007000,0.000000,...,6.5,7.0,17.4,8.7,4.6,11.7,NaN,10.6,33412.48,NaN
9,2011,01001,NaN,53944.0,0.515000,0.789000,0.181000,0.002000,0.007000,0.000000,...,6.2,7.1,18.3,8.2,4.4,11.3,NaN,10.9,32795.85,NaN


In [62]:
print(df[(df['fips']=='01001')]['unemployment_rate'])
print(df[(df['fips']=='01001')]['employed_persons'])
df[(df['fips']=='01001')]['total_pop']

0     4.8
1     5.0
2     4.7
3     3.8
4     3.3
5     3.4
6     5.3
7     8.9
8     4.0
9     4.8
10    5.4
11    5.9
12    5.3
13    7.6
14    5.6
15    5.2
16    4.2
17    3.7
18    2.9
Name: unemployment_rate, dtype: float64
0     21096.0
1     21557.0
2     22146.0
3     22986.0
4     23619.0
5     23610.0
6     23376.0
7     22464.0
8         NaN
9         NaN
10        NaN
11        NaN
12        NaN
13        NaN
14        NaN
15        NaN
16        NaN
17        NaN
18        NaN
Name: employed_persons, dtype: float64


0     45909.0
1     46800.0
2     48366.0
3     49676.0
4     51328.0
5     52405.0
6     53277.0
7     54135.0
8     53155.0
9     53944.0
10    54590.0
11    54907.0
12    55136.0
13    55221.0
14    55049.0
15    55036.0
16    55200.0
17    55380.0
18    55639.0
Name: total_pop, dtype: float64

In [81]:
for i in df.index:
    if df.loc[i, 'year'] <=2009 :
        df.loc[i, 'employment_pop_ratio'] = df.loc[i,'employed_persons']/df.loc[i,'total_pop'] * 100
df['employment_pop_ratio'].head(20)

0     45.951774
1     46.061966
2     45.788364
3     46.271842
4     46.015820
5     45.052953
6     43.876344
7     41.496259
8     60.400000
9     58.700000
10    57.700000
11    56.700000
12    56.800000
13    56.000000
14    56.800000
15    56.100000
16    55.600000
17    55.800000
18    55.400000
19    44.527126
Name: employment_pop_ratio, dtype: float64

In [83]:
df.drop(columns = 'employed_persons', inplace = True)

In [96]:
counts = df['fips'].value_counts()
ys = df[df['fips'].isin(counts[counts == 19].index)]

ys.head(10)

,year,fips,covid_cases,total_pop,pct_female,pct_white,pct_black,pct_native_american,pct_asian,pct_native_hawaiian_pacific_islander,...,pct_ind_information,pct_ind_finance_and_insurance_and_real_estate_and_rental_and_leasing,pct_ind_professional_scientific_and_management_and_administrative_and_waste_management_services,pct_ind_educational_services_and_health_care_and_social_assistance,pct_ind_arts_entertainment_and_recreation_and_accommodation_and_food_services,pct_ind_other_services_except_public_administration,pct_ind_public_administration,health_insurance_rate,poverty_rate,per_capita_income_2022
0,2002,01001,NaN,45909.0,0.512906,0.807445,0.172123,0.004792,0.006055,0.000283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.3,70687.65
1,2003,01001,NaN,46800.0,0.513675,0.805897,0.172842,0.004808,0.006282,0.000321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.4,71228.01
2,2004,01001,NaN,48366.0,0.513873,0.804139,0.173531,0.004673,0.006740,0.000351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.6,70791.24
3,2005,01001,NaN,49676.0,0.512843,0.802440,0.173706,0.004549,0.007247,0.000745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.4,68428.88
4,2006,01001,NaN,51328.0,0.513170,0.799388,0.176181,0.004481,0.007286,0.000760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5,67876.86
5,2007,01001,NaN,52405.0,0.514016,0.797328,0.177273,0.004446,0.007709,0.000725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.4,71532.50
6,2008,01001,NaN,53277.0,0.514368,0.795071,0.178126,0.004730,0.008033,0.000845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7,69689.70
7,2009,01001,NaN,54135.0,0.512995,0.793091,0.179071,0.004674,0.008497,0.000868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.2,72720.97
8,2010,01001,NaN,53155.0,0.515000,0.791000,0.179000,0.003000,0.007000,0.000000,...,1.1,6.5,7.0,17.4,8.7,4.6,11.7,NaN,10.6,33412.48
9,2011,01001,NaN,53944.0,0.515000,0.789000,0.181000,0.002000,0.007000,0.000000,...,1.0,6.2,7.1,18.3,8.2,4.4,11.3,NaN,10.9,32795.85


In [98]:
ys.to_csv('../Data/Final/STI0220_cov0220.csv')